**Group Members**\
        **Cherukuri Nikhilesh - S20180010040**\
        **Kore Nithish Kumar - S20180010086**\
        **Pulla Nagendra Babu - S20180010138**\
        **Rishab Tripati - S20180010147**

In [ ]:
import pandas as pd
import statistics
import numpy as np 
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.model_selection import KFold ,RepeatedKFold,train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
import seaborn as sns
from scipy.stats import shapiro,pearsonr
from scipy import stats
import scipy as sp
from sklearn.metrics import r2_score
from statsmodels.graphics.gofplots import qqplot
from statsmodels.stats.stattools import durbin_watson
from sklearn import preprocessing,metrics,datasets, linear_model,svm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn import linear_model,tree

In [ ]:
!pip install factor_analyzer==0.2.3

In [ ]:
sheets=pd.read_excel('/content/sample_data/stock portfolio performance data set.xlsx',sheet_name=['all period'],skiprows=[0], usecols = [1,2,3,4,5,6,13,14,15,16,17,18])
df=pd.concat(sheets[frame] for frame in sheets.keys())
df.keys()
df.describe()

In [ ]:
df.isnull().values.any()

In [ ]:
df.head()

In [ ]:
X= df.drop(['Annual Return.1', 'Excess Return.1','Systematic Risk.1', 'Total Risk.1', 'Abs. Win Rate.1','Rel. Win Rate.1'],axis=1)
Y = df.drop([" Large B/P "," Large ROE "," Large S/P "," Large Return Rate in the last quarter "," Large Market Value "," Small systematic Risk"],axis=1)
for each in X.keys():
    qqplot(X[each],line='s')
    plt.show()

In [ ]:
boxplot = X.boxplot(grid=False,rot=45, fontsize=9)

In [ ]:
df.shape

In [ ]:
z = np.abs(stats.zscore(df))
df_o = df[(z < 3).all(axis=1)]
print(df_o.shape)

In [ ]:
X= df_o.drop(['Annual Return.1', 'Excess   Return.1','Systematic Risk.1', 'Total Risk.1', 'Abs. Win Rate.1','Rel. Win Rate.1'],axis=1)
Y = df_o.drop([" Large B/P "," Large ROE "," Large S/P "," Large Return Rate in the last quarter "," Large Market Value "," Small systematic Risk"],axis=1)
boxplot = X.boxplot(grid=False,rot=45, fontsize=9)

In [ ]:
correlation=df_o.corr()
print(correlation)

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(correlation, annot=True, cmap='coolwarm')

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(whiten=True)
pca.fit(X)
variance = pd.DataFrame(pca.explained_variance_ratio_)
print(variance)
cumulative=np.cumsum(pca.explained_variance_ratio_)
print(cumulative)

In [ ]:
df3 = pd.DataFrame({'variance_explained':cumulative,
             'PC':['PC1','PC2','PC3','PC4','PC5','PC6']})
sns.barplot(x='PC',y="variance_explained", data=df3, color="c");

In [ ]:
df2 = pd.DataFrame({'var':pca.explained_variance_ratio_,
             'PC':['PC1','PC2','PC3','PC4','PC5','PC6']})
sns.barplot(x='PC',y="var", data=df2, color="c");

In [ ]:
components=pd.DataFrame(pca.components_,columns=X.columns,index = ['PC-1','PC-2','PC-3','PC-4','PC-5','PC-6'])
components.head(6)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y,test_size=0.2,random_state=1)

In [ ]:
targets=pd.DataFrame(columns=['Annual Return.1', 'Excess Return.1','Systematic Risk.1', 'Total Risk.1', 'Abs. Win Rate.1','Rel. Win Rate.1'],index=y_test.index.values) 
for y in targets.keys():
    reg = linear_model.LinearRegression() 
    reg.fit(x_train, y_train[y])
    print("\n")
    print("model for",y,'evaluation parameter')
    print("The linear model is: Y = {:.5} + {:.5}*large b/p + {:.5}*large ROE + {:.5}*large s/p+ {:.5}*large return rates+ {:.5}*large market sales+ {:.5}*small system risk".format(reg.intercept_, reg.coef_[0], reg.coef_[1], reg.coef_[2],reg.coef_[3],reg.coef_[4],reg.coef_[5]))
    print('Variance score: {}'.format(reg.score(x_test, y_test[y]))) 
    y_pred = reg.predict(x_test)
    targets[y]= y_pred          
    fig, ax = plt.subplots(1,1)
    sns.regplot(x=y_pred, y=y_test[y], lowess=True, ax=ax, line_kws={'color': 'red'})
    ax.set_title('Observed vs. Predicted Values', fontsize=16)
    ax.set(xlabel='Predicted', ylabel='Observed')


In [ ]:
from yellowbrick.regressor import ResidualsPlot

In [ ]:
for y in targets.keys():
    model_ols = sm.OLS(y_train[y],x_train).fit()
    print(model_ols.summary())

In [ ]:
for y in targets.keys():
    for x in X.keys():
        colors = (0,0,0)
        area = np.pi*3
        df_o.plot.scatter(x=x, y=y)

In [ ]:
error_list=[]
for y in targets.keys():
    error = y_test[y] - targets[y]
    error_info = pd.DataFrame({'y_true': y_test[y], 'y_pred': targets[y], 'error': error}, columns=['y_true', 'y_pred', 'error'])
    error_list.append(error_info)
    plt.figure(figsize=(8,5))
    g = sns.scatterplot(x="y_pred", y="error", data=error_info, color='blue')
    g.set_title(f'Check Homoskedasticity {y}', fontsize=15)
    g.set_xlabel("predicted values", fontsize=13)
    g.set_ylabel("Residual", fontsize=13)

In [ ]:
for error in error_list: 
      fig, ax = plt.subplots(figsize=(8,5))
      ax = error.error.plot()
      dw=durbin_watson(error.error,axis=0)
      print(dw)
      ax.set_title('Uncorrelated errors', fontsize=15)
      ax.set_xlabel("Data", fontsize=13)
      ax.set_ylabel("Residual", fontsize=13)

In [ ]:
for error in error_list:
    fig, ax = plt.subplots(figsize=(6,4))
    _ = sp.stats.probplot(error.error, plot=ax, fit=True)
    ax.set_title('Probability plot', fontsize=15)
    ax.set_xlabel("Theoritical Qunatiles", fontsize=13)
    ax.set_ylabel("Ordered Values", fontsize=13)
    ax = sm.qqplot(error.error, line='45')
    plt.show()

In [ ]:
def coefficient_of_determination(ys_orig,ys_line):
    y_mean_line = [statistics.mean(ys_orig) for y in ys_orig]
    squared_error_regr = mean_squared_error(ys_orig, ys_line)
    squared_error_y_mean = mean_squared_error(ys_orig, y_mean_line)
    return 1 - (squared_error_regr/squared_error_y_mean)
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
def mean_squared_error(y_true,y_pred):
    return  metrics.mean_squared_error(y_true, y_pred)
def goodness(y_true, y_pred):
    mape = mean_absolute_percentage_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)

In [ ]:
for y in targets.keys():
    r_squared = coefficient_of_determination(y_test[y],targets[y])
    print(r_squared)

In [ ]:
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
chi_square_value,p_value=calculate_bartlett_sphericity(X)
chi_square_value, p_value

In [ ]:
from factor_analyzer import FactorAnalyzer

In [ ]:
# Create factor analysis object and perform factor analysis
fa = FactorAnalyzer()
fa.analyze(X,6, rotation=None)
# Check Eigenvalues
ev, v = fa.get_eigenvalues()
ev 

In [ ]:
# Create scree plot using matplotlib
plt.scatter(range(1,X.shape[1]+1),ev)
plt.plot(range(1,X.shape[1]+1),ev)
plt.title('Scree Plot')
plt.xlabel('Factors')
plt.ylabel('Eigenvalue')
plt.grid()
plt.show()

In [ ]:
fa.loadings

In [ ]:
# Get variance of each factors
fa.get_factor_variance()